In [1]:
import random
import pandas as pd
import numpy as np 
import seaborn as sns
from tensorflow import keras
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import preprocessing 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import PoissonRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

2023-12-03 12:40:27.104125: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-03 12:40:27.137334: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-03 12:40:27.137958: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-03 12:40:27.697219: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# ASSOCIATION RULES

In [2]:
# ! pip install apyori

In [3]:
df = pd.read_csv('Groceries_dataset.csv')
df.head()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Member_number    38765 non-null  int64 
 1   Date             38765 non-null  object
 2   itemDescription  38765 non-null  object
dtypes: int64(1), object(2)
memory usage: 908.7+ KB


In [5]:
df_count = df.groupby(['itemDescription']).size().reset_index(name='count')
df_count

,itemDescription,count
0,Instant food products,60
1,UHT-milk,323
2,abrasive cleaner,22
3,artif. sweetener,29
4,baby cosmetics,3
...,...,...
162,white bread,362
163,white wine,176
164,whole milk,2502
165,yogurt,1334


In [6]:
grocery_df = df.groupby('Date').agg({'itemDescription':','.join})
grocery_df

,itemDescription
Date,
01-01-2014,"cleaner,sausage,tropical fruit,whole milk,citr..."
01-01-2015,"canned beer,frozen meals,butter,ham,citrus fru..."
01-02-2014,"other vegetables,pip fruit,frankfurter,sugar,o..."
01-02-2015,"whole milk,canned beer,bottled water,soda,whol..."
01-03-2014,"sausage,meat,sugar,sausage,bottled beer,whole ..."
...,...
31-07-2015,"vinegar,chicken,root vegetables,chicken,sausag..."
31-08-2014,"pork,other vegetables,beef,whole milk,hygiene ..."
31-08-2015,"brown bread,frankfurter,chicken,citrus fruit,b..."


In [7]:
grocery_df['itemDescription'] = grocery_df['itemDescription'].str.split(',')
grocery_df

,itemDescription
Date,
01-01-2014,"[cleaner, sausage, tropical fruit, whole milk,..."
01-01-2015,"[canned beer, frozen meals, butter, ham, citru..."
01-02-2014,"[other vegetables, pip fruit, frankfurter, sug..."
01-02-2015,"[whole milk, canned beer, bottled water, soda,..."
01-03-2014,"[sausage, meat, sugar, sausage, bottled beer, ..."
...,...
31-07-2015,"[vinegar, chicken, root vegetables, chicken, s..."
31-08-2014,"[pork, other vegetables, beef, whole milk, hyg..."
31-08-2015,"[brown bread, frankfurter, chicken, citrus fru..."


In [8]:
transactions=[]
for i in range(0,len(grocery_df)):
    transactions.append(grocery_df['itemDescription'][i])

In [9]:
from apyori import apriori
rules = apriori(transactions=transactions,min_support=0.01,min_confidence=0.8,min_lift=3,
                min_length=3,max_length=3)

In [10]:
rules

<generator object apriori at 0x7f283ebd8040>

In [11]:
results = list(rules)
len(results)

4

In [12]:
def inspect(results):
    baseItem   = [tuple(result[2][0][0])[0:] for result in results]
    addItem     = [tuple(result[2][0][1])[0] for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(baseItem,addItem, supports, confidences, lifts))
resultsinDataFrame = pd.DataFrame(inspect(results),columns=['BaseItems','AdditionalItems','Supports',
                                                            'Confidences','Lifts'])

In [13]:
resultsinDataFrame

,BaseItems,AdditionalItems,Supports,Confidences,Lifts
0,"(coffee, artif. sweetener)",specialty bar,0.010989,0.800000,3.165217
1,"(frozen dessert, rice)",butter milk,0.010989,1.000000,3.192982
2,"(napkins, male cosmetics)",cat food,0.010989,0.800000,3.757419
3,"(chicken, prosecco)",hygiene articles,0.012363,0.818182,3.290809
